# myLoguru

In [2]:
# loguru与logging一样使用logger对象
# 相较于logging, loguru更加强大且便捷
from loguru import logger

## 基本用法

### 分级

In [3]:
# loguru共有七个等级
# 分别为：trace, debug, info, success, warning, error, critical
def guru():
    logger.trace('trace msg')
    logger.debug('debug msg')
    logger.info('info msg')
    logger.success('success msg')
    logger.warning('warning msg')
    logger.error('error msg')
    logger.critical('critical msg')
guru()

2025-08-20 14:38:36.980 | DEBUG    | __main__:guru:5 - debug msg
2025-08-20 14:38:36.982 | INFO     | __main__:guru:6 - info msg
2025-08-20 14:38:36.982 | SUCCESS  | __main__:guru:7 - success msg
2025-08-20 14:38:36.983 | WARNING  | __main__:guru:8 - warning msg
2025-08-20 14:38:36.984 | ERROR    | __main__:guru:9 - error msg
2025-08-20 14:38:36.984 | CRITICAL | __main__:guru:10 - critical msg


### remove和add

In [4]:
# .remove()用于清除默认设置（如默认在console输出）
logger.remove()
# .add()是loguru中一个强大的方法，可以用于添加自定义设定
# 注意：add只会“添加”设定，而非“替换”。如需，可使用remove清除原本的设定
logger.add('msg.log', level="ERROR") # level指定显示等级
guru()

In [5]:
# 此外,.add()会返回对应的handle_id，可使用remove(handle_id)进行移除
logger.remove()
import sys  # sys.stdout指定logger在console中输出
handle_id = logger.add(sys.stdout, level="ERROR") # level指定显示等级
# 移除设定后，这段代码什么都不会做
logger.remove(handle_id)
guru()

#### 日志文件设置

In [6]:
logger.remove()
# 此外，当sink为文件时（即将日志保存到文件，则可以对日志文件进行设定）
# rotation设置文件大小，当超过该大小时会自动新建立日志文件，并以日期时间重命名旧文件
# retention设置文件保存天数，在运行时会清理过期日志
# compression设置旧日志文件压缩格式（只会压缩旧日志文件，最新的日志文件仍会是文本格式）
logger.add('rotation.log', rotation="50 KB", retention="1 minutes", compression="zip")
for i in range(1000):
    logger.debug('debug msg')
logger.critical('critical msg')

## 高级用法

### 控制日志格式

In [ ]:
logger.remove()
# 自定义日志输出格式及颜色
logger.add(sys.stdout, format='<level>{time}</level> - <RED>{level}</RED> - <YELLOW>{message}</YELLOW>')
guru()

2025-08-20T14:38:37.110696+0800 - DEBUG - debug msg
2025-08-20T14:38:37.111693+0800 - INFO - info msg
2025-08-20T14:38:37.111693+0800 - SUCCESS - success msg
2025-08-20T14:38:37.112691+0800 - WARNING - warning msg
2025-08-20T14:38:37.112691+0800 - ERROR - error msg
2025-08-20T14:38:37.113687+0800 - CRITICAL - critical msg


In [ ]:
logger.add(
    sys.stdout,
    format="{time:YYYY-MM-DD HH:mm:ss} - {level} - {message}",
    level="INFO"
)
logger.info("Custom format log")

{"text": "2025-08-20T19:25:27.714801+0800 - INFO - Custom format log - {}\n", "record": {"elapsed": {"repr": "4:46:50.752708", "seconds": 17210.752708}, "exception": null, "extra": {}, "file": {"name": "317142773.py", "path": "C:\\Users\\eric3\\AppData\\Local\\Temp\\ipykernel_12308\\317142773.py"}, "function": "<module>", "level": {"icon": "ℹ️", "name": "INFO", "no": 20}, "line": 6, "message": "Custom format log", "module": "317142773", "name": "__main__", "process": {"id": 12308, "name": "MainProcess"}, "thread": {"id": 16640, "name": "MainThread"}, "time": {"repr": "2025-08-20 19:25:27.714801+08:00", "timestamp": 1755689127.714801}}}
2025-08-20 19:25:27 | INFO | Custom format log


### 自定义属性（子logger)

In [8]:
logger.remove()
# {extra}用于记录自定义属性
logger.add(sys.stdout, format='{time} - {level} - <level>{message}</level> - {extra}')
child = logger.bind(foo='bar', hello='world')
child.info('child logger')

2025-08-20T14:38:37.120670+0800 - INFO - child logger - {'foo': 'bar', 'hello': 'world'}


#### 上下文管理器

In [9]:
with logger.contextualize(programmer="want"):
    guru()

2025-08-20T14:38:37.136626+0800 - DEBUG - debug msg - {'programmer': 'want'}
2025-08-20T14:38:37.137624+0800 - INFO - info msg - {'programmer': 'want'}
2025-08-20T14:38:37.138621+0800 - SUCCESS - success msg - {'programmer': 'want'}
2025-08-20T14:38:37.138621+0800 - WARNING - warning msg - {'programmer': 'want'}
2025-08-20T14:38:37.139618+0800 - ERROR - error msg - {'programmer': 'want'}
2025-08-20T14:38:37.139618+0800 - CRITICAL - critical msg - {'programmer': 'want'}


#### 装饰器

In [ ]:
@logger.contextualize(foo='bar')
def test():
    logger.info('info msg')
    logger.warning('warning msg')
test()

2025-08-20T14:38:37.152584+0800 - INFO - info msg - {'foo': 'bar'}
2025-08-20T14:38:37.153582+0800 - WARNING - warning msg - {'foo': 'bar'}


### JSON结构化

In [21]:
logger.remove()
# serialize参数用于控制日志的JSON结构化
logger.add(
    sys.stdout,
    serialize=True,
    format="{time} - {level} - {message} - {extra}"
)
logger_json = logger.bind(data={'key': 'value'}) # 可将字典作为extra传入
logger_json.info("Structured log")

{"text": "2025-08-20T19:20:58.295227+0800 - INFO - Structured log - {'data': {'key': 'value'}}\n", "record": {"elapsed": {"repr": "4:42:21.333134", "seconds": 16941.333134}, "exception": null, "extra": {"data": {"key": "value"}}, "file": {"name": "4287663844.py", "path": "C:\\Users\\eric3\\AppData\\Local\\Temp\\ipykernel_12308\\4287663844.py"}, "function": "<module>", "level": {"icon": "ℹ️", "name": "INFO", "no": 20}, "line": 9, "message": "Structured log", "module": "4287663844", "name": "__main__", "process": {"id": 12308, "name": "MainProcess"}, "thread": {"id": 16640, "name": "MainThread"}, "time": {"repr": "2025-08-20 19:20:58.295227+08:00", "timestamp": 1755688858.295227}}}


### 异常处理

#### 手动捕获异常

In [11]:
logger.remove()
logger.add(sys.stdout)
logger.add('except.log')
try:
    1 / 0   # 除0异常
except:
    # loguru会记录异常的完整信息
    logger.exception('get except')

2025-08-20 14:38:37.169 | ERROR    | __main__:<module>:8 - get except
Traceback (most recent call last):

  File "C:\Users\eric3\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
           │         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack...
           │         └ <code object <module> at 0x000002D678A73050, file "e:\Study\python\loguru_study\.venv\lib\site-packages\ipykernel_launcher.py...
           └ <function _run_code at 0x000002D678A6F1C0>

  File "C:\Users\eric3\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack...
         └ <code object <module> at 0x000002D678A73050, file "e:\Study\python\loguru_study\.ven

#### 自动记录异常

In [12]:
# logger.catch()可以自动捕获异常
with logger.catch():
    1 / 0

2025-08-20 14:38:37.199 | ERROR    | __main__:<module>:2 - An error has been caught in function '<module>', process 'MainProcess' (12308), thread 'MainThread' (16640):
Traceback (most recent call last):

  File "C:\Users\eric3\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
           │         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack...
           │         └ <code object <module> at 0x000002D678A73050, file "e:\Study\python\loguru_study\.venv\lib\site-packages\ipykernel_launcher.py...
           └ <function _run_code at 0x000002D678A6F1C0>

  File "C:\Users\eric3\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack.

In [ ]:
# logger.catch()默认捕获所有类型的异常
# 可以通过传递参数自定义捕获异常的类型，并设置异常的日志等级
with logger.catch(ZeroDivisionError, level="WARNING"):
    1 / 0

2025-08-20 14:38:37.213 | WARNING  | __main__:<module>:3 - An error has been caught in function '<module>', process 'MainProcess' (12308), thread 'MainThread' (16640):
Traceback (most recent call last):

  File "C:\Users\eric3\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
           │         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack...
           │         └ <code object <module> at 0x000002D678A73050, file "e:\Study\python\loguru_study\.venv\lib\site-packages\ipykernel_launcher.py...
           └ <function _run_code at 0x000002D678A6F1C0>

  File "C:\Users\eric3\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack.

In [15]:
# 此外，logger.catch()也可作为装饰器
@logger.catch()
def test():
    1 / 0
test()

2025-08-20 14:39:37.534 | ERROR    | __main__:<module>:5 - An error has been caught in function '<module>', process 'MainProcess' (12308), thread 'MainThread' (16640):
Traceback (most recent call last):

  File "C:\Users\eric3\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
           │         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack...
           │         └ <code object <module> at 0x000002D678A73050, file "e:\Study\python\loguru_study\.venv\lib\site-packages\ipykernel_launcher.py...
           └ <function _run_code at 0x000002D678A6F1C0>

  File "C:\Users\eric3\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack.